In [73]:
import numpy as np
import pandas as pd
from google.cloud import bigquery
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.express as px


def orders_by_vendors():

    query = f"""
    select  region
       ,global_entity_id
       ,country_code
       ,country_name
       ,city
       ,date_trunc(partition_date_local,MONTH) month
       ,coalesce(chain_name, vendor_name) vendor
       ,count (distinct order_id) total_orders
from (
SELECT
  o.global_entity_id
  ,o.country_code
  ,c.country_name
  ,c.region
  ,o.partition_date_local
  ,o.delivery_location.city_english city
  ,o.order_id
  ,o.vendor_id
  ,items.name item_name
  ,items.quantity
  ,items.unit_price_eur
  ,v.name vendor_name
  ,v.chain.name chain_name
  ,payment.payment_method
FROM
  `fulfillment-dwh-production.curated_data_shared_coredata_business.orders` o , UNNEST(items) AS items
  left join  `fulfillment-dwh-production.cl._all_vendors` v on v.vendor_code = o.vendor_id and o.global_entity_id = v.entity_id 
  left join `fulfillment-dwh-production.curated_data_shared.countries`c on c.country_iso = o.country_code
  where partition_date_local between '2024-06-01' and '2024-06-30'
  --and o.global_entity_id = 'MJM_AT'
  and o.vertical_type = 'restaurants'
  --and o.order_id = 'j6es-2423-175e'
)
where city is not null
group by 1,2,3,4,5,6,7
order by global_entity_id,country_code,city, total_orders desc

    """

    return query


def top_n_vendors():
   
  query = f"""
   select  *
  from (
  SELECT
          c.region,
          o.global_entity_id,
          o.country_code,
          c.country_name,
          DATE_TRUNC(o.partition_date_local, MONTH) AS month,
          coalesce(v.chain.name, v.name) vendor,
          COUNT(DISTINCT o.order_id) AS total_orders,
          ROW_NUMBER() OVER (
              PARTITION BY c.region, o.global_entity_id, o.country_code, c.country_name
              ORDER BY COUNT(DISTINCT o.order_id) DESC
          ) AS rank
  FROM
    `fulfillment-dwh-production.curated_data_shared_coredata_business.orders` o , UNNEST(items) AS items
    left join  `fulfillment-dwh-production.cl._all_vendors` v on v.vendor_code = o.vendor_id and o.global_entity_id = v.entity_id 
    left join `fulfillment-dwh-production.curated_data_shared.countries`c on c.country_iso = o.country_code
    WHERE
          o.partition_date_local BETWEEN '2024-07-01' AND '2024-07-30'
          AND o.vertical_type = 'restaurants'
          AND o.delivery_location.city_english IS NOT NULL
      GROUP BY
          c.region,
          o.global_entity_id,
          o.country_code,
          c.country_name,
          MONTH,
          vendor
  )
  where rank <= 20
  order by region, country_name, total_orders desc,rank
  """

  return query


def top_n_items_city():
   
  query = f"""
    select  *
  from (
  SELECT
          c.region,
          o.global_entity_id,
          o.country_code,
          c.country_name,
          DATE_TRUNC(o.partition_date_local, MONTH) AS month,
          o.delivery_location.city_english AS city,
          items.name AS item_name,
          COUNT(DISTINCT o.order_id) AS total_orders,
          ROW_NUMBER() OVER (
              PARTITION BY c.region, o.global_entity_id, o.country_code, c.country_name, o.delivery_location.city_english
              ORDER BY COUNT(DISTINCT o.order_id) DESC
          ) AS rank
  FROM
    `fulfillment-dwh-production.curated_data_shared_coredata_business.orders` o , UNNEST(items) AS items
    left join  `fulfillment-dwh-production.cl._all_vendors` v on v.vendor_code = o.vendor_id and o.global_entity_id = v.entity_id 
    left join `fulfillment-dwh-production.curated_data_shared.countries`c on c.country_iso = o.country_code
    WHERE
          o.partition_date_local BETWEEN '2024-07-01' AND '2024-07-30'
          AND o.vertical_type = 'restaurants'
          AND o.delivery_location.city_english IS NOT NULL
      GROUP BY
          c.region,
          o.global_entity_id,
          o.country_code,
          c.country_name,
          DATE_TRUNC(o.partition_date_local, MONTH),
          city,
          item_name
  )
  where rank <= 20
  and total_orders > 500
  order by region, country_name, city, total_orders desc,rank

  """
   

  return query 



def top_n_items_country():
   
  query = f"""
   
  select  *
  from (
  SELECT
          c.region,
          o.global_entity_id,
          o.country_code,
          c.country_name,
          DATE_TRUNC(o.partition_date_local, MONTH) AS month,
          items.name AS item_name,
          COUNT(DISTINCT o.order_id) AS total_orders,
          ROW_NUMBER() OVER (
              PARTITION BY c.region, o.global_entity_id, o.country_code, c.country_name
              ORDER BY COUNT(DISTINCT o.order_id) DESC
          ) AS rank
  FROM
    `fulfillment-dwh-production.curated_data_shared_coredata_business.orders` o , UNNEST(items) AS items
    left join  `fulfillment-dwh-production.cl._all_vendors` v on v.vendor_code = o.vendor_id and o.global_entity_id = v.entity_id 
    left join `fulfillment-dwh-production.curated_data_shared.countries`c on c.country_iso = o.country_code
    WHERE
          o.partition_date_local BETWEEN '2024-07-01' AND '2024-07-30'
          AND o.vertical_type = 'restaurants'
          AND o.delivery_location.city_english IS NOT NULL
      GROUP BY
          c.region,
          o.global_entity_id,
          o.country_code,
          c.country_name,
          DATE_TRUNC(o.partition_date_local, MONTH),
          item_name
  )
  where rank <= 20
  and total_orders > 500
  order by region, country_name, total_orders desc,rank

  """
   
  return query


def display_vendors_country(df, region):

    lst = df[df['region'] == region]['country_name'].unique()

    for i in lst:
        display(df[df['country_name'] == i])

In [74]:
# define a few things (project id, start date etc.)
project_id = "logistics-customer-staging"
client = bigquery.Client(project = project_id)

#total_orders = client.query(orders_by_vendors()).to_dataframe()
top_vendors = client.query(top_n_vendors()).to_dataframe()

#top_items_city = client.query(items_n_items()).to_dataframe()
#top_items_country = client.query(top_n_items_country()).to_dataframe()

/Users/shazeb.asad/global_pricing/.venv/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/shazeb.asad/global_pricing/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [ ]:
pd.set_option("display.max_rows", None)

In [ ]:
orders_by_region['values_var'] / df.groupby('group_var')['values_var'].transform('sum')

In [81]:
orders_by_region = total_orders.groupby(['region','month']).agg({'total_orders' : 'sum'}).reset_index()
orders_by_region = orders_by_region.sort_values(by=['total_orders'], ascending=[False])
orders_by_region['percentage_of_total'] = (orders_by_region['total_orders'] /  orders_by_region['total_orders'].sum()) * 100
display(orders_by_region)

orders_by_country = total_orders.groupby(['region','global_entity_id','country_name','month']).agg({'total_orders' : 'sum'}).reset_index()
orders_by_country = orders_by_country.sort_values(by=['region','total_orders','country_name'], ascending=[False,False, True])
orders_by_country['percentage_of_region'] = (orders_by_country['total_orders'] / orders_by_country.groupby('region')['total_orders'].transform('sum')
) * 100
display(orders_by_country)

display_vendors_country(top_vendors, 'Europe')
display_vendors_country(top_vendors, 'Asia')
display_vendors_country(top_vendors, 'Americas')
display_vendors_country(top_vendors, 'MENA')

,region,month,total_orders,percentage_of_total
3,MENA,2024-06-01,54904115,47.560677
1,Asia,2024-06-01,29598612,25.639791
0,Americas,2024-06-01,18242227,15.802325
2,Europe,2024-06-01,12695191,10.997206


,region,global_entity_id,country_name,month,total_orders,percentage_of_region
34,MENA,HS_SA,Saudi Arabia,2024-06-01,12096382,22.031831
42,MENA,YS_TR,Turkey,2024-06-01,10306842,18.77244
35,MENA,TB_AE,United Arab Emirates,2024-06-01,9368079,17.062617
39,MENA,TB_KW,Kuwait,2024-06-01,7618115,13.875308
33,MENA,HF_EG,Egypt,2024-06-01,5326006,9.700559
41,MENA,TB_QA,Qatar,2024-06-01,2877654,5.241236
40,MENA,TB_OM,Oman,2024-06-01,2062792,3.757081
38,MENA,TB_JO,Jordan,2024-06-01,1962957,3.575246
36,MENA,TB_BH,Bahrain,2024-06-01,1799760,3.278006
37,MENA,TB_IQ,Iraq,2024-06-01,1485528,2.705677


,region,global_entity_id,country_code,country_name,month,vendor,total_orders,rank
520,Europe,MJM_AT,AT,Austria,2024-07-01,McDonald's,148846,1
521,Europe,MJM_AT,AT,Austria,2024-07-01,Noodle King,31192,2
522,Europe,MJM_AT,AT,Austria,2024-07-01,Burger King,18414,3
523,Europe,MJM_AT,AT,Austria,2024-07-01,KFC,13822,4
524,Europe,MJM_AT,AT,Austria,2024-07-01,L'Osteria,11857,5
525,Europe,MJM_AT,AT,Austria,2024-07-01,Subway,10989,6
526,Europe,MJM_AT,AT,Austria,2024-07-01,Honu Tiki Bowl,10773,7
527,Europe,MJM_AT,AT,Austria,2024-07-01,Le Burger,10114,8
528,Europe,MJM_AT,AT,Austria,2024-07-01,Vapiano,9571,9
529,Europe,MJM_AT,AT,Austria,2024-07-01,Starbucks,8490,10


,region,global_entity_id,country_code,country_name,month,vendor,total_orders,rank
540,Europe,DJ_CZ,CZ,Czech Republic,2024-07-01,McDonald's,124183,1
541,Europe,DJ_CZ,CZ,Czech Republic,2024-07-01,KFC Rozvoz,39633,2
542,Europe,DJ_CZ,CZ,Czech Republic,2024-07-01,Bageterie Boulevard,23374,3
543,Europe,DJ_CZ,CZ,Czech Republic,2024-07-01,Restaurace Globus,15071,4
544,Europe,DJ_CZ,CZ,Czech Republic,2024-07-01,Chachaři,14386,5
545,Europe,DJ_CZ,CZ,Czech Republic,2024-07-01,Turbo Pizza,11792,6
546,Europe,DJ_CZ,CZ,Czech Republic,2024-07-01,Burger King,8699,7
547,Europe,DJ_CZ,CZ,Czech Republic,2024-07-01,Rud's Pizza,8471,8
548,Europe,DJ_CZ,CZ,Czech Republic,2024-07-01,UGO Salaterie,7711,9
549,Europe,DJ_CZ,CZ,Czech Republic,2024-07-01,Los Capolitos,7177,10


,region,global_entity_id,country_code,country_name,month,vendor,total_orders,rank
560,Europe,PO_FI,FI,Finland,2024-07-01,Hesburger,89565,1
561,Europe,PO_FI,FI,Finland,2024-07-01,McDonald's,35877,2
562,Europe,PO_FI,FI,Finland,2024-07-01,Pizza hut,7344,3
563,Europe,PO_FI,FI,Finland,2024-07-01,Boneless,7325,4
564,Europe,PO_FI,FI,Finland,2024-07-01,Burger Company,6826,5
565,Europe,PO_FI,FI,Finland,2024-07-01,Kebab Keisarit Group,3977,6
566,Europe,PO_FI,FI,Finland,2024-07-01,Subway,2460,7
567,Europe,PO_FI,FI,Finland,2024-07-01,Dif döner,1649,8
568,Europe,PO_FI,FI,Finland,2024-07-01,Da Max,1598,9
569,Europe,PO_FI,FI,Finland,2024-07-01,Chicken Joint,1351,10


...

,region,global_entity_id,country_code,country_name,month,vendor,total_orders,rank
760,MENA,TB_OM,OM,Oman,2024-07-01,KFC,60701,1
761,MENA,TB_OM,OM,Oman,2024-07-01,McDonald's,42588,2
762,MENA,TB_OM,OM,Oman,2024-07-01,Tea Time,39130,3
763,MENA,TB_OM,OM,Oman,2024-07-01,ChicKing,26846,4
764,MENA,TB_OM,OM,Oman,2024-07-01,Kucu,25778,5
765,MENA,TB_OM,OM,Oman,2024-07-01,RFC,20600,6
766,MENA,TB_OM,OM,Oman,2024-07-01,Boom Burger & Sweets,18462,7
767,MENA,TB_OM,OM,Oman,2024-07-01,My Berry,18328,8
768,MENA,TB_OM,OM,Oman,2024-07-01,Papa Johns,17671,9
769,MENA,TB_OM,OM,Oman,2024-07-01,Mango Talaat,15435,10


,region,global_entity_id,country_code,country_name,month,vendor,total_orders,rank
780,MENA,TB_QA,QA,Qatar,2024-07-01,McDonald's,188874,1
781,MENA,TB_QA,QA,Qatar,2024-07-01,KFC,87171,2
782,MENA,TB_QA,QA,Qatar,2024-07-01,Tea Time,57265,3
783,MENA,TB_QA,QA,Qatar,2024-07-01,Papa John's,57083,4
784,MENA,TB_QA,QA,Qatar,2024-07-01,Jollibee,39152,5
785,MENA,TB_QA,QA,Qatar,2024-07-01,Karak Mqanes,32867,6
786,MENA,TB_QA,QA,Qatar,2024-07-01,Go Crispy Restaurant,31231,7
787,MENA,TB_QA,QA,Qatar,2024-07-01,Hardee's,24890,8
788,MENA,TB_QA,QA,Qatar,2024-07-01,Starbucks,22838,9
789,MENA,TB_QA,QA,Qatar,2024-07-01,Pizza Hut,21259,10


,region,global_entity_id,country_code,country_name,month,vendor,total_orders,rank
800,MENA,HS_SA,SA,Saudi Arabia,2024-07-01,McDonald's KSA,346731,1
801,MENA,HS_SA,SA,Saudi Arabia,2024-07-01,Al Baik,344994,2
802,MENA,HS_SA,SA,Saudi Arabia,2024-07-01,KFC,271828,3
803,MENA,HS_SA,SA,Saudi Arabia,2024-07-01,Shawarma House,267074,4
804,MENA,HS_SA,SA,Saudi Arabia,2024-07-01,Dunkin Donuts,214061,5
805,MENA,HS_SA,SA,Saudi Arabia,2024-07-01,McDonald's,212299,6
806,MENA,HS_SA,SA,Saudi Arabia,2024-07-01,Shawarmer,181187,7
807,MENA,HS_SA,SA,Saudi Arabia,2024-07-01,Kudu,159247,8
808,MENA,HS_SA,SA,Saudi Arabia,2024-07-01,Herfy,155587,9
809,MENA,HS_SA,SA,Saudi Arabia,2024-07-01,DEEP FRIES,126207,10


,region,global_entity_id,country_code,country_name,month,vendor,total_orders,rank
820,MENA,YS_TR,TR,Turkey,2024-07-01,Burger King,656911,1
821,MENA,YS_TR,TR,Turkey,2024-07-01,Komagene Etsiz Çiğ Köfte,471177,2
822,MENA,YS_TR,TR,Turkey,2024-07-01,Domino's Pizza,351590,3
823,MENA,YS_TR,TR,Turkey,2024-07-01,McDonald's,323962,4
824,MENA,YS_TR,TR,Turkey,2024-07-01,Popeyes,282898,5
825,MENA,YS_TR,TR,Turkey,2024-07-01,Maydonoz Döner,265135,6
826,MENA,YS_TR,TR,Turkey,2024-07-01,Starbucks Coffee,240082,7
827,MENA,YS_TR,TR,Turkey,2024-07-01,Öncü Döner,177867,8
828,MENA,YS_TR,TR,Turkey,2024-07-01,KFC,173831,9
829,MENA,YS_TR,TR,Turkey,2024-07-01,Köfteci Yusuf,152255,10


,region,global_entity_id,country_code,country_name,month,vendor,total_orders,rank
840,MENA,TB_AE,AE,United Arab Emirates,2024-07-01,McDonald's,682723,1
841,MENA,TB_AE,AE,United Arab Emirates,2024-07-01,KFC,364962,2
842,MENA,TB_AE,AE,United Arab Emirates,2024-07-01,Pizza Hut,204924,3
843,MENA,TB_AE,AE,United Arab Emirates,2024-07-01,Hardee's,171261,4
844,MENA,TB_AE,AE,United Arab Emirates,2024-07-01,Starbucks,168911,5
845,MENA,TB_AE,AE,United Arab Emirates,2024-07-01,ALBAIK,158084,6
846,MENA,TB_AE,AE,United Arab Emirates,2024-07-01,Burger King,157422,7
847,MENA,TB_AE,AE,United Arab Emirates,2024-07-01,Subway,95772,8
848,MENA,TB_AE,AE,United Arab Emirates,2024-07-01,Baskin Robbins,80090,9
849,MENA,TB_AE,AE,United Arab Emirates,2024-07-01,Texas Chicken,79952,10
